In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
data = pd.read_csv('C:\\Users\\hkimr\\Desktop\\REU Day 1\\Aggregated User Data.csv')
data = data[(data['UUID'] == '0BFC35E2-4817-4865-BFA7-764742302A2D') | (data['UUID'] == '0A986513-7828-4D53-AA1F-E02D6DF9561B') | (data['UUID'] == '00EABED2-271D-49D8-B599-1D4A09240601')] 
data.drop(columns = ['timestamp'], inplace = True)

In [4]:
def interpolation(df):
    col_to_avg = list(df.columns) #Start with keeping all the columns as columns to use an average interpolation on
    for k in range(len(list(df.columns))):
        if list(df.columns)[k].startswith(('discrete', 'label')): #Remove label and discrete columns from col_to_avg
            col_to_avg.remove(list(df.columns)[k])
    
    df_with_avg = df[col_to_avg].fillna(df[col_to_avg].mean()) #Interpolate nan columns for all continuous-valued columns with average
    
    col_to_zero = list(df.columns)
    for k in range(len(list(df.columns))):
        if not list(df.columns)[k].startswith(('discrete', 'label')): #Remove all columns except label and discrete
            col_to_zero.remove(list(df.columns)[k])
    
    df_with_zero = df[col_to_zero].fillna(0) #Interpolate nan values for label and discrete columns with 0
    
    return pd.concat([df_with_avg, df_with_zero], axis = 1)

In [5]:
data.iloc[:,1:27] = interpolation(data.iloc[:,1:27])
data = data.iloc[:,:27].values
data

array([['00EABED2-271D-49D8-B599-1D4A09240601', 0.996815, 0.003529, ...,
        0.10692, 0.516842, 0.255494],
       ['00EABED2-271D-49D8-B599-1D4A09240601', 0.996864, 0.004172, ...,
        -0.079483, 0.357748, 0.036252],
       ['00EABED2-271D-49D8-B599-1D4A09240601', 0.996825, 0.003667, ...,
        0.122432, 0.464881, 0.222375],
       ...,
       ['0BFC35E2-4817-4865-BFA7-764742302A2D', 1.012776, 0.008135, ...,
        -0.099042, -0.307551, -0.592356],
       ['0BFC35E2-4817-4865-BFA7-764742302A2D', 1.013261, 0.013788, ...,
        0.03825, -0.251999, -0.714921],
       ['0BFC35E2-4817-4865-BFA7-764742302A2D', 1.004676,
        0.016819999999999998, ..., 0.566422, -0.23814, -0.670271]],
      dtype=object)

In [6]:
X = data[:, 1:]
y = data[:, 0]

y = LabelEncoder().fit_transform(y)

In [7]:
# ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [0])], remainder = 'passthrough')
# data = np.array(ct.fit_transform(data))
# data

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[ 0.26613015, -0.24255077, -0.24541348, ...,  1.14760479,
         0.02512358, -1.35435394],
       [ 1.54782469,  2.64174475,  2.48155211, ..., -0.90346572,
        -1.93681673,  1.0396404 ],
       [-0.45782768, -0.41033278, -0.37066374, ..., -0.1459623 ,
        -0.03468168,  0.32183197],
       ...,
       [-0.44728588, -0.41023807, -0.37039906, ..., -0.10584491,
        -0.06240541, -0.14067355],
       [-0.33844089, -0.41202802, -0.37553758, ...,  0.0072894 ,
         0.09502979,  0.13329722],
       [-0.37424388, -0.40057801, -0.35470056, ...,  1.32873031,
        -0.1814447 ,  0.13470702]])

In [9]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = 26):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 3) 
        )
    def forward(self, x):
        return self.network(x)

In [10]:
model = Classifier().to(device)
lr = 0.01
n_epochs = 2000
batch_size = 1000

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train).to(device)
train_labels = torch.tensor(y_train).to(device)
test_features = torch.tensor(X_test).to(device)
test_labels = torch.tensor(y_test).to(device)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [11]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        #_, encoded_labels = torch.max(labels, dim = 1) 
        
        optimizer.zero_grad()
        preds = model(features.float())
        loss = criterion(preds, labels.long())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 8.452657580375671, Final Batch Loss: 1.0029115676879883
Epoch 2, Loss: 6.810325920581818, Final Batch Loss: 0.7941502928733826
Epoch 3, Loss: 5.141149699687958, Final Batch Loss: 0.5626130700111389
Epoch 4, Loss: 4.5037864446640015, Final Batch Loss: 0.6755846738815308
Epoch 5, Loss: 4.1495475471019745, Final Batch Loss: 0.5890994071960449
Epoch 6, Loss: 3.929561495780945, Final Batch Loss: 0.4637000858783722
Epoch 7, Loss: 3.7704228460788727, Final Batch Loss: 0.4398846924304962
Epoch 8, Loss: 3.5692365765571594, Final Batch Loss: 0.4171736240386963
Epoch 9, Loss: 3.4975081384181976, Final Batch Loss: 0.4982609152793884
Epoch 10, Loss: 3.3161933720111847, Final Batch Loss: 0.4064342975616455
Epoch 11, Loss: 3.175704061985016, Final Batch Loss: 0.4102758467197418
Epoch 12, Loss: 2.9627137780189514, Final Batch Loss: 0.381295382976532
Epoch 13, Loss: 2.8468790352344513, Final Batch Loss: 0.29292115569114685
Epoch 14, Loss: 2.8992702662944794, Final Batch Loss: 0.366980284

Epoch 113, Loss: 1.2907700538635254, Final Batch Loss: 0.15705634653568268
Epoch 114, Loss: 1.2480899840593338, Final Batch Loss: 0.16529279947280884
Epoch 115, Loss: 1.2381225228309631, Final Batch Loss: 0.14548958837985992
Epoch 116, Loss: 1.224033311009407, Final Batch Loss: 0.13309933245182037
Epoch 117, Loss: 1.2820281609892845, Final Batch Loss: 0.12076505273580551
Epoch 118, Loss: 1.2211118340492249, Final Batch Loss: 0.15790601074695587
Epoch 119, Loss: 1.2312316000461578, Final Batch Loss: 0.1836921125650406
Epoch 120, Loss: 1.2649518698453903, Final Batch Loss: 0.17261449992656708
Epoch 121, Loss: 1.2473984360694885, Final Batch Loss: 0.14820876717567444
Epoch 122, Loss: 1.185902178287506, Final Batch Loss: 0.15543974936008453
Epoch 123, Loss: 1.1987377405166626, Final Batch Loss: 0.1197771430015564
Epoch 124, Loss: 1.2591908127069473, Final Batch Loss: 0.15214157104492188
Epoch 125, Loss: 1.1433367431163788, Final Batch Loss: 0.13686317205429077
Epoch 126, Loss: 1.3238283544

Epoch 224, Loss: 1.0532724931836128, Final Batch Loss: 0.16509877145290375
Epoch 225, Loss: 1.1560942456126213, Final Batch Loss: 0.12412422150373459
Epoch 226, Loss: 1.0511873960494995, Final Batch Loss: 0.13520415127277374
Epoch 227, Loss: 1.1522658318281174, Final Batch Loss: 0.1097395271062851
Epoch 228, Loss: 1.1148828342556953, Final Batch Loss: 0.1376066654920578
Epoch 229, Loss: 1.148111715912819, Final Batch Loss: 0.159621000289917
Epoch 230, Loss: 1.162608988583088, Final Batch Loss: 0.16812582314014435
Epoch 231, Loss: 1.0840385258197784, Final Batch Loss: 0.1369127780199051
Epoch 232, Loss: 1.1536090224981308, Final Batch Loss: 0.16646786034107208
Epoch 233, Loss: 1.121617965400219, Final Batch Loss: 0.14308501780033112
Epoch 234, Loss: 1.0690984055399895, Final Batch Loss: 0.11170210689306259
Epoch 235, Loss: 1.1221139580011368, Final Batch Loss: 0.13782547414302826
Epoch 236, Loss: 1.2583446353673935, Final Batch Loss: 0.19950512051582336
Epoch 237, Loss: 1.16947226226329

Epoch 336, Loss: 1.0453571304678917, Final Batch Loss: 0.12302788347005844
Epoch 337, Loss: 1.0988544002175331, Final Batch Loss: 0.1352972388267517
Epoch 338, Loss: 1.0546266064047813, Final Batch Loss: 0.11204594373703003
Epoch 339, Loss: 0.9990298077464104, Final Batch Loss: 0.16901282966136932
Epoch 340, Loss: 1.0582939758896828, Final Batch Loss: 0.13218268752098083
Epoch 341, Loss: 1.0423812866210938, Final Batch Loss: 0.11720585823059082
Epoch 342, Loss: 1.0595249459147453, Final Batch Loss: 0.11846496164798737
Epoch 343, Loss: 1.0485378354787827, Final Batch Loss: 0.1547873467206955
Epoch 344, Loss: 1.1207054927945137, Final Batch Loss: 0.14625614881515503
Epoch 345, Loss: 1.0320797860622406, Final Batch Loss: 0.13867603242397308
Epoch 346, Loss: 1.090537965297699, Final Batch Loss: 0.1601693034172058
Epoch 347, Loss: 1.073759414255619, Final Batch Loss: 0.211301788687706
Epoch 348, Loss: 0.9953962787985802, Final Batch Loss: 0.15987230837345123
Epoch 349, Loss: 1.0348018556833

Epoch 446, Loss: 1.0239691659808159, Final Batch Loss: 0.14892391860485077
Epoch 447, Loss: 0.9897215068340302, Final Batch Loss: 0.12575778365135193
Epoch 448, Loss: 1.0514966174960136, Final Batch Loss: 0.15258054435253143
Epoch 449, Loss: 0.9766040369868279, Final Batch Loss: 0.09219025075435638
Epoch 450, Loss: 1.1071361973881721, Final Batch Loss: 0.13836687803268433
Epoch 451, Loss: 1.0191571712493896, Final Batch Loss: 0.09898212552070618
Epoch 452, Loss: 0.9975694939494133, Final Batch Loss: 0.1470990628004074
Epoch 453, Loss: 1.0765556395053864, Final Batch Loss: 0.126869797706604
Epoch 454, Loss: 0.9681954383850098, Final Batch Loss: 0.0912022516131401
Epoch 455, Loss: 0.9939718320965767, Final Batch Loss: 0.1136603131890297
Epoch 456, Loss: 0.9546384289860725, Final Batch Loss: 0.10142140090465546
Epoch 457, Loss: 1.0322430282831192, Final Batch Loss: 0.12261504679918289
Epoch 458, Loss: 1.0333571806550026, Final Batch Loss: 0.13317625224590302
Epoch 459, Loss: 1.08001839369

...

Epoch 1542, Loss: 0.9185132533311844, Final Batch Loss: 0.09179750084877014
Epoch 1543, Loss: 0.8898975029587746, Final Batch Loss: 0.11949580907821655
Epoch 1544, Loss: 0.9207122251391411, Final Batch Loss: 0.11309270560741425
Epoch 1545, Loss: 0.8711458742618561, Final Batch Loss: 0.08552693575620651
Epoch 1546, Loss: 0.9545991718769073, Final Batch Loss: 0.10459583252668381
Epoch 1547, Loss: 0.8323897570371628, Final Batch Loss: 0.09451115131378174
Epoch 1548, Loss: 0.8125688508152962, Final Batch Loss: 0.07517699897289276
Epoch 1549, Loss: 0.8841305151581764, Final Batch Loss: 0.09946753084659576
Epoch 1550, Loss: 0.8883554935455322, Final Batch Loss: 0.12808862328529358
Epoch 1551, Loss: 0.8442421481013298, Final Batch Loss: 0.10169752687215805
Epoch 1552, Loss: 0.9227398782968521, Final Batch Loss: 0.14783526957035065
Epoch 1553, Loss: 0.8506432846188545, Final Batch Loss: 0.09528203308582306
Epoch 1554, Loss: 0.8272336646914482, Final Batch Loss: 0.12383288890123367
Epoch 1555, 

Epoch 1651, Loss: 0.8244170248508453, Final Batch Loss: 0.08726482093334198
Epoch 1652, Loss: 0.8729513511061668, Final Batch Loss: 0.11888913065195084
Epoch 1653, Loss: 0.8335856199264526, Final Batch Loss: 0.10790140181779861
Epoch 1654, Loss: 0.8678343147039413, Final Batch Loss: 0.12247569859027863
Epoch 1655, Loss: 0.8806635662913322, Final Batch Loss: 0.1263911873102188
Epoch 1656, Loss: 0.9340362921357155, Final Batch Loss: 0.08661210536956787
Epoch 1657, Loss: 0.8886002823710442, Final Batch Loss: 0.09577710926532745
Epoch 1658, Loss: 1.017842710018158, Final Batch Loss: 0.11847478896379471
Epoch 1659, Loss: 0.8610674738883972, Final Batch Loss: 0.11421968787908554
Epoch 1660, Loss: 0.8851964101195335, Final Batch Loss: 0.1183246299624443
Epoch 1661, Loss: 0.8687025979161263, Final Batch Loss: 0.09104790538549423
Epoch 1662, Loss: 0.9675706699490547, Final Batch Loss: 0.12357139587402344
Epoch 1663, Loss: 0.8792935982346535, Final Batch Loss: 0.12056159228086472
Epoch 1664, Los

Epoch 1760, Loss: 0.8841630220413208, Final Batch Loss: 0.15259985625743866
Epoch 1761, Loss: 0.8512294143438339, Final Batch Loss: 0.1283458173274994
Epoch 1762, Loss: 0.9149079620838165, Final Batch Loss: 0.1648966521024704
Epoch 1763, Loss: 0.9182878956198692, Final Batch Loss: 0.15725719928741455
Epoch 1764, Loss: 0.8876120075583458, Final Batch Loss: 0.11356841772794724
Epoch 1765, Loss: 0.8720181211829185, Final Batch Loss: 0.06732376664876938
Epoch 1766, Loss: 0.909199595451355, Final Batch Loss: 0.10152260959148407
Epoch 1767, Loss: 0.9126124083995819, Final Batch Loss: 0.04653090983629227
Epoch 1768, Loss: 0.8502082601189613, Final Batch Loss: 0.1293950080871582
Epoch 1769, Loss: 0.8922435119748116, Final Batch Loss: 0.12649869918823242
Epoch 1770, Loss: 0.8205600827932358, Final Batch Loss: 0.11711743474006653
Epoch 1771, Loss: 0.8270266205072403, Final Batch Loss: 0.08740878850221634
Epoch 1772, Loss: 0.8586643487215042, Final Batch Loss: 0.09923919290304184
Epoch 1773, Loss

Epoch 1870, Loss: 0.8447941914200783, Final Batch Loss: 0.09458165615797043
Epoch 1871, Loss: 0.841091550886631, Final Batch Loss: 0.1042364314198494
Epoch 1872, Loss: 0.8359026834368706, Final Batch Loss: 0.10391608625650406
Epoch 1873, Loss: 0.8272225260734558, Final Batch Loss: 0.08976110070943832
Epoch 1874, Loss: 0.84597547352314, Final Batch Loss: 0.09416525065898895
Epoch 1875, Loss: 0.9117953926324844, Final Batch Loss: 0.12200742959976196
Epoch 1876, Loss: 0.8349642604589462, Final Batch Loss: 0.10573839396238327
Epoch 1877, Loss: 1.1195251196622849, Final Batch Loss: 0.1200401782989502
Epoch 1878, Loss: 0.8637277632951736, Final Batch Loss: 0.12627342343330383
Epoch 1879, Loss: 0.8523919731378555, Final Batch Loss: 0.09634555131196976
Epoch 1880, Loss: 0.8699508532881737, Final Batch Loss: 0.10121600329875946
Epoch 1881, Loss: 0.8996635973453522, Final Batch Loss: 0.08448123186826706
Epoch 1882, Loss: 0.9066944345831871, Final Batch Loss: 0.09640615433454514
Epoch 1883, Loss:

Epoch 1980, Loss: 0.803814023733139, Final Batch Loss: 0.09197128564119339
Epoch 1981, Loss: 0.8559728488326073, Final Batch Loss: 0.10158321261405945
Epoch 1982, Loss: 0.8112792670726776, Final Batch Loss: 0.11827623099088669
Epoch 1983, Loss: 0.8329750522971153, Final Batch Loss: 0.11575019359588623
Epoch 1984, Loss: 0.8587446957826614, Final Batch Loss: 0.10512000322341919
Epoch 1985, Loss: 0.8259455040097237, Final Batch Loss: 0.07814937829971313
Epoch 1986, Loss: 0.8278924897313118, Final Batch Loss: 0.10927694290876389
Epoch 1987, Loss: 0.8585933595895767, Final Batch Loss: 0.15377414226531982
Epoch 1988, Loss: 0.9216277524828911, Final Batch Loss: 0.19521872699260712
Epoch 1989, Loss: 0.8969684541225433, Final Batch Loss: 0.10693397372961044
Epoch 1990, Loss: 0.839872270822525, Final Batch Loss: 0.11297328770160675
Epoch 1991, Loss: 0.8860588073730469, Final Batch Loss: 0.09893977642059326
Epoch 1992, Loss: 0.9027882516384125, Final Batch Loss: 0.0838257372379303
Epoch 1993, Los

In [14]:
softmax = nn.Softmax(dim = 1)
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    #_, answers = torch.max(labels, dim = 1)
    print(metrics.confusion_matrix(labels.cpu(), preds.cpu()))
    print(metrics.classification_report(labels.cpu(), preds.cpu(), digits = 3))

[[405  47   7]
 [ 27 734  14]
 [  5   9 623]]
              precision    recall  f1-score   support

           0      0.927     0.882     0.904       459
           1      0.929     0.947     0.938       775
           2      0.967     0.978     0.973       637

    accuracy                          0.942      1871
   macro avg      0.941     0.936     0.938      1871
weighted avg      0.942     0.942     0.941      1871



In [15]:
torch.save(model.state_dict(), f"../saved_models/2000Epochs_0.01LR_UserClassifier")